In [1]:
import numpy as np
import sys,os
sys.path.append('../LIB/')
from env import ENV
from tqdm import tqdm
import pandas as pd

# binary search

In [167]:
pv = 470790
length = 12
payment = 47041

def calculate_payment(pv,length,rate):
    return rate *pv / (1- (1+rate)**(-length))

def calculate_rate(pv,length,payment,space=np.arange(0,1,0.00001)):
    assert length * payment >= pv
    target = payment
    low = 0
    up = len(space)
    space_len = len(space)
    while up - low > 1:
        cur_index = int((low+up) / 2)

        rate = space[cur_index]
        cal = calculate_payment(pv,length,rate)
        if cal == target:
            return rate
        elif cal < target:
            low = cur_index
        else:
            up = cur_index
    if up == space_len:
        up -= 1
    up_cal = calculate_payment(pv,length,space[up])
    low_cal = calculate_payment(pv,length,space[low])
    if abs(up_cal - target) <= abs(low_cal - target):
        return space[up]
    else:
        return space[low]
        

# Load files 

In [109]:
pre_ori = pd.read_csv(ENV.previous_application_ori.value)
print(pre_ori.shape)

# ins_ori = pd.read_csv(ENV.installments_payments_ori.value)
# print(ins_ori.shape)

# pos_ori = pd.read_csv(ENV.POS_CASH_balance_ori.value)
# print(pos_ori.shape)

# app_ori = pd.read_csv(ENV.application_train_ori.value)
# print(app_ori.shape)
# app_ori_test = pd.read_csv(ENV.application_test_ori.value)
# print(app_ori.shape)

# X = pd.concat([app_ori.drop('TARGET',axis=1),app_ori_test])
# print(X.shape)

(1670214, 37)


## Get Loan Type

In [4]:
X.NAME_CONTRACT_TYPE.value_counts()

Cash loans         326537
Revolving loans     29718
Name: NAME_CONTRACT_TYPE, dtype: int64

In [130]:
X_cash = X[X.NAME_CONTRACT_TYPE=='Cash loans'][['AMT_CREDIT','AMT_ANNUITY','SK_ID_CURR']].copy()

## View N/A

In [131]:
X_cash.isnull().sum()

AMT_CREDIT      0
AMT_ANNUITY    36
SK_ID_CURR      0
dtype: int64

In [132]:
print('train')
print(app_ori[app_ori.NAME_CONTRACT_TYPE=='Cash loans'].AMT_ANNUITY.isnull().sum())
print('test')
print(app_ori_test[app_ori_test.NAME_CONTRACT_TYPE=='Cash loans'].AMT_ANNUITY.isnull().sum())

train
12
test
24


In [133]:
print(X_cash.shape)
print('after drop na')
X_cash = X_cash.dropna()
print(X_cash.shape)

(326537, 3)
after drop na
(326501, 3)


# Step 1. add new col lowLimitLength

In [134]:
X_cash['lowLimitLength'] = np.ceil(X_cash.AMT_CREDIT.values/X_cash.AMT_ANNUITY.values)
X_cash['lowLimitLength'] = X_cash['lowLimitLength'].astype('int')

In [135]:
pv =135000
n = 12
payment = 16020
calculate_rate(pv,n,payment)

0.059070000000000004

In [136]:
pv =500490.0
n = 12
payment = 56731.5
calculate_rate(pv,n,payment)

0.05083000000000001

### for those have length 9.

In [29]:
X_cash_9 = X_cash[X_cash.lowLimitLength==9].copy()
pv_9 = X_cash_9.AMT_CREDIT.values
payment_9 = X_cash_9.AMT_ANNUITY.values
rate_9 = []
for index in range(len(X_cash_9)):
    pv = pv_9[index]
    p = payment_9[index]
    n = 12
    rate_9.append(calculate_rate(pv,n,p))
X_cash_9['apr'] = rate_9
    

### for those have length x, and approximate length q

In [ ]:
low_limit_length = [9,]
map2l = [12,]
counts = [7153,]
min_l = [0.04991,]
max_l = [0.06780,]

In [46]:
x = 9
q = 9

X_cash_x = X_cash[X_cash.lowLimitLength==x].copy()
pv_x = X_cash_x.AMT_CREDIT.values
payment_x = X_cash_x.AMT_ANNUITY.values
rate_x = []
for index in range(len(X_cash_x)):
    pv = pv_x[index]
    p = payment_x[index]
    n = q
    rate_x.append(calculate_rate(pv,n,p))
X_cash_x['apr'] = rate_x

In [60]:
pv =58500.0
n = 120
payment = 6597.0
calculate_rate(pv,n,payment)

0.11277000000000001

In [181]:
def estimate_terms(pv,p,ns,t_apr=0.03,nu=60,step =6):
    space_start = int(ns + step - ns %step)
    space = list(range(space_start,nu+1,step))
    low = 0
    up = len(space)
    space_len = len(space)
    while up - low > 1:
        cur_index = int((low+up) / 2)
        cur_n = space[cur_index]
        cal_apr = calculate_rate(pv,cur_n,p)
        if cal_apr == t_apr:
            return cal_apr,cur_n
        elif cal_apr < t_apr:
            low = cur_index
        elif cal_apr > t_apr:
            up = cur_index
    if up == space_len:
        up -= 1
    up_cal = calculate_rate(pv,space[up],p)
    low_cal = calculate_rate(pv,space[low],p)
    if abs(up_cal - t_apr) <= abs(low_cal - t_apr):
        return up_cal,space[up]
    else:
        return low_cal,space[low]

In [175]:
print(pv)
print(p)
print(ns)

406597.5
24700.5
17


## Rough Estimate

In [173]:
X_cash

,AMT_CREDIT,AMT_ANNUITY,SK_ID_CURR,lowLimitLength
0,406597.5,24700.5,100002,17
1,1293502.5,35698.5,100003,37
3,312682.5,29686.5,100006,11
4,513000.0,21865.5,100007,24
5,490495.5,27517.5,100008,18
6,1560726.0,41301.0,100009,38
7,1530000.0,42075.0,100010,37
8,1019610.0,33826.5,100011,31
10,652500.0,21177.0,100014,31
11,148365.0,10678.5,100015,14


In [197]:

pvs = X_cash.AMT_CREDIT.values
payments = X_cash.AMT_ANNUITY.values
low_lengths = X_cash.lowLimitLength.values
rates = []
terms = []
for index in tqdm(range(len(X_cash))):
    pv = pvs[index]
    p = payments[index]
    ns = low_lengths[index]
    r,l = estimate_terms(pv,p,ns,t_apr=0.025)
    rates.append(r)
    terms.append(l)
X_cash['estimate_apr'] = rates
X_cash['estimate_terms'] = terms

100%|██████████| 326501/326501 [00:28<00:00, 11385.20it/s]


In [ ]:
X_cash[['SK_ID_CURR']]

In [200]:
X_cash.to_pickle('../../data/add_features/aprANDtermsv1.pkl')

In [199]:
X_cash

,AMT_CREDIT,AMT_ANNUITY,SK_ID_CURR,lowLimitLength,estimate_apr,estimate_terms
0,406597.5,24700.5,100002,17,0.03266,24
1,1293502.5,35698.5,100003,37,0.01830,60
3,312682.5,29686.5,100006,11,0.02066,12
4,513000.0,21865.5,100007,24,0.02526,36
5,490495.5,27517.5,100008,18,0.02531,24
6,1560726.0,41301.0,100009,38,0.01662,60
7,1530000.0,42075.0,100010,37,0.01816,60
8,1019610.0,33826.5,100011,31,0.02392,54
10,652500.0,21177.0,100014,31,0.02514,60
11,148365.0,10678.5,100015,14,0.02880,18
